# **Codificación Huffman**
Este cuaderno interactivo presenta una exploración completa del **algoritmo de codificación de Huffman**, desde su implementación fundamental hasta su análisis de rendimiento en diversos escenarios. El objetivo es demostrar la eficacia y versatilidad de esta técnica de compresión sin pérdida.

El contenido está estructurado en tres secciones principales:

1.  **El Motor de Compresión:** En esta primera parte, se definen todas las funciones esenciales que componen nuestro compresor. Aquí se encuentra la lógica para analizar archivos, aplicar el algoritmo de Huffman, calcular métricas de compresión y generar las visualizaciones para el análisis.

2.  **Aplicaciones Prácticas:** Se demuestra el uso del algoritmo en casos concretos. Exploramos cómo se comporta al comprimir diferentes tipos de archivos, como un **texto literario** extraído del Quijote y un archivo de **código fuente** en Python, destacando las diferencias en sus resultados.

3.  **Experimentos y Análisis de Rendimiento:** Finalmente, se somete el algoritmo a una serie de pruebas para evaluar su eficiencia. Se analiza su rendimiento en distintos escenarios (datos normales, repetitivos y aleatorios) y se estudia su **escalabilidad** al enfrentarse a archivos de tamaños crecientes.

### Importación de librerías
En esta celda se importan las librerías que se usan en el notebook.
Además se define la variable *project_root* en la cual se almacena el directorio raíz del proyecto, ya que este notebook está bajo el directorio *notebooks/* lo cual le limita el acceso a los demás archivos.

In [ ]:
import os
import pickle
import sys
from collections import Counter
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random 
import string 
# Aqui se añade la carpeta root del proyecto, ya que se esta trabajando en notebook/
project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if project_root not in sys.path:
    sys.path.append(project_root)

## Funciones del encoder
### Función Principal de Compresión y Análisis
Esta función es el núcleo del motor. Recibe la ruta de un archivo, lo comprime y devuelve un diccionario con las estadísticas del proceso.

In [ ]:
from encoder.compressor_huffman import CompresorHuffman

def analizar_compresion(ruta_archivo):
    """
    Toma la ruta de un archivo, lo comprime con Huffman y devuelve
    un diccionario con las estadísticas del proceso.
    """
    compresor = CompresorHuffman()
    
    # Manejo de excepción, por si no encuentra el archivo
    try:
        with open(ruta_archivo, 'r', encoding='utf-8') as f:
            texto = f.read()
        # Se obtiene el tamaño del archivo antes de la compresión
        tamano_original_bytes = os.path.getsize(ruta_archivo)
    except FileNotFoundError:
        print(f"Error: El archivo {ruta_archivo} no fue encontrado.")
        return None

    # Se realiza la compresión
    texto_comprimido, raiz = compresor.compresion_huffman(texto)

    # Aqui se calcula el tamaño de los archivos para el posterior analisis
    longitud_en_bits = len(texto_comprimido)
    tamano_datos_comprimidos_bytes = (longitud_en_bits + 7) // 8
    
    arbol_serializado = pickle.dumps(raiz)
    tamano_arbol_bytes = len(arbol_serializado)
    
    tamano_total_comprimido_bytes = tamano_datos_comprimidos_bytes + tamano_arbol_bytes
    
    # Se calcula el ratio  = tamaño despues / tamaño antes
    if tamano_original_bytes > 0:
        ratio_compresion = (1 - (tamano_total_comprimido_bytes / tamano_original_bytes)) * 100
    else:
        ratio_compresion = 0

    """
    Se devuelven los resultados para la comprensi[on en forma de diccionario.
    Esto para su posterior analisis
    """
    return {
        "archivo": os.path.basename(ruta_archivo),
        "original_b": tamano_original_bytes,
        "total_comprimido_b": tamano_total_comprimido_bytes,
        "ratio_%": ratio_compresion
    }


### Función para Procesar Lotes de Archivos
Esta utilidad toma una lista de archivos, los procesa usando la función anterior y muestra un resumen comparativo en formato de tabla.

In [ ]:
def obtener_resultados(archivos_a_probar):
    """
    aqui se definen los archivos y se obtienen los resultados.
    """
    # Lista de archivos para los exp

    lista_de_resultados = []
    
    for archivo in archivos_a_probar:
        resultado = analizar_compresion(archivo)
        if resultado:
            lista_de_resultados.append(resultado)


    # Resumen
    print("\n" + "="*60)
    print("Resumen de Resultados de Compresión Huffman")
    print("="*60)
    print(f"{'Archivo':<25} | {'Original (B)':>12} | {'Comprimido (B)':>15} | {'Ratio (%)':>10}")
    print("-"*60)
    
    for res in lista_de_resultados:
        print(f"{res['archivo']:<25} | {res['original_b']:>12} | {res['total_comprimido_b']:>15} | {res['ratio_%']:>9.2f}%")
        
    print("="*60)

    return lista_de_resultados
    
# Ejemplo del formato que debe seguir la lista de archivos a probar:
archivos_a_probar = [
        '../data/quijote.txt',
        '../data/texto_repetitivo.txt',
        '../data/texto_aleatorio.txt'
    ]

## Funciones para analisis de datos
### Utilidad: Gráfico Comparativo de Tamaños
Esta función genera un gráfico de barras que compara el tamaño original contra el tamaño comprimido de los archivos procesados.

In [ ]:
def crear_grafico_comparativo(resultados):
    """
    Crea un gráfico de barras agrupadas comparando el tamaño
    original vs el comprimido para cada archivo.
    """
    # Extraer los datos para graficar
    nombres_archivos = [res['archivo'] for res in resultados]
    tamanos_originales = [res['original_b'] for res in resultados]
    tamanos_comprimidos = [res['total_comprimido_b'] for res in resultados]
    
    # Configuración del gráfico
    x = np.arange(len(nombres_archivos))  # las etiquetas ubicaciones
    ancho_barra = 0.35  # ancho de barras

    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(figsize=(12, 7))

    # crear las barras
    rects1 = ax.bar(x - ancho_barra/2, tamanos_originales, ancho_barra, label='Original', color='#4c72b0')
    rects2 = ax.bar(x + ancho_barra/2, tamanos_comprimidos, ancho_barra, label='Comprimido', color='#c44e52')

    # Aanadir títulos y etiquetas
    ax.set_title('Comparación de Tamaño: Original vs. Comprimido', fontsize=16, weight='bold')
    ax.set_ylabel('Tamaño (bytes)', fontsize=12)
    ax.set_xlabel('Archivos de Prueba', fontsize=12)
    ax.set_xticks(x)
    ax.set_xticklabels(nombres_archivos, rotation=15, ha="right")
    ax.legend()

    # añadir etiquetas de valor sobre las barras para mayor claridad
    ax.bar_label(rects1, padding=3)
    ax.bar_label(rects2, padding=3)

    plt.tight_layout()

    if not os.path.exists('../results'):
        os.makedirs('results')
    plt.savefig('../results/grafico_comparativo_tamanos.png')
    plt.show()


### Utilidad: Generador de Datos para Pruebas de Escalabilidad
Esta función crea archivos de texto de diferentes tamaños para poder analizar cómo escala el rendimiento del algoritmo.

In [ ]:
def preparar_datos_escalabilidad(tipo='normal', tamaños_kb=[1, 5, 10, 20, 50]):
    """
    Genera archivos de prueba de diferentes tamaños para un tipo de texto específico.
    Tipos: 'normal', 'repetitivo', 'aleatorio'
    """
    # Cargar el texto base
    if tipo == 'normal':
        with open('../data/quijote.txt', 'r', encoding='utf-8') as f:
            texto_base = f.read()
    elif tipo == 'repetitivo':
        texto_base = "abcde" * 20000 # ~100KB de texto repetitivo
    else: # aleatorio
        texto_base = ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(100 * 1024))

    # Crear sub-archivos de diferentes tamaños
    if not os.path.exists('../data/temp'):
        os.makedirs('../data/temp')

    for kb in tamaños_kb:
        longitud = kb * 1024
        contenido = (texto_base * (longitud // len(texto_base) + 1))[:longitud]
        with open(f'../data/temp/{tipo}_{kb}kb.txt', 'w', encoding='utf-8') as f:
            f.write(contenido)

### Utilidad: Gráfico de Análisis de Escalabilidad
Finalmente, esta función visualiza los resultados de las pruebas de escalabilidad, mostrando la relación entre el tamaño original y el comprimido.

In [ ]:
def graficar_escalabilidad(df_resultados):
    """
    Crea un gráfico de líneas mostrando cómo escala el tamaño comprimido
    respecto al tamaño original para diferentes tipos de archivos.
    """
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(figsize=(12, 8))

    # Línea base de "sin compresión" (y=x)
    max_size = df_resultados['original_b'].max()
    ax.plot([0, max_size], [0, max_size], 'k--', label='Sin Compresión (y=x)')

    # Un color para cada tipo de archivo
    colores = {'normal': '#4c72b0', 'repetitivo': '#55a868', 'aleatorio': '#c44e52'}

    # Agrupar por tipo y graficar una línea para cada uno
    for tipo, grupo in df_resultados.groupby('Tipo'):
        ax.plot(grupo['original_b'], grupo['total_comprimido_b'], marker='o', linestyle='-', label=f'Texto {tipo}', color=colores[tipo])

    # Títulos y etiquetas
    ax.set_title('Análisis de Escalabilidad de la Compresión Huffman', fontsize=16, weight='bold')
    ax.set_xlabel('Tamaño Original (bytes)', fontsize=12)
    ax.set_ylabel('Tamaño Comprimido (bytes)', fontsize=12)
    ax.legend()
    ax.grid(True)

    plt.tight_layout()
    plt.savefig('../results/grafico_escalabilidad.png')
    plt.show()


# Aplicaciones Prácticas

## Aplicación 1: Compresor de Archivos de Texto (.txt)

La aplicación más clásica y fundamental de la codificación de Huffman es la **compresión de archivos de texto**. En idiomas como el español o el inglés, la frecuencia de los caracteres no es uniforme: algunas letras como la 'e' o la 'a' y el carácter de espacio son extremadamente comunes, mientras que otras como la 'k', 'w' o 'x' son muy raras.

El algoritmo de Huffman explota esta característica asignando **códigos binarios más cortos a los caracteres más frecuentes** y códigos más largos a los menos frecuentes. Esto resulta en una representación del texto que ocupa significativamente menos espacio que el formato estándar (como ASCII o UTF-8), donde cada carácter tiene una longitud fija (generalmente 8 bits).

A continuación, aplicaremos nuestro compresor a un extracto del libro "Don Quijote de la Mancha" para demostrar su eficacia.

In [ ]:

# Definimos la ruta del archivo de texto que vamos a comprimir.
# Usamos el mismo que en el experimento para mantener la consistencia.
ruta_quijote = '../data/quijote.txt'

print("="*60)
print("APLICACIÓN 1: ANÁLISIS DE COMPRESIÓN DE TEXTO")
print("="*60)

# Utilizamos la función principal para analizar la compresión del archivo.
resultado_texto = analizar_compresion(ruta_quijote)

if resultado_texto:
    # Imprimimos los resultados de una manera clara y descriptiva.
    print(f"Archivo Analizado:   {resultado_texto['archivo']}")
    print(f"Tamaño Original:      {resultado_texto['original_b']} bytes")
    print(f"Tamaño Comprimido:    {resultado_texto['total_comprimido_b']} bytes")
    print(f"Ratio de Compresión:  {resultado_texto['ratio_%']:.2f}%")

else:
    print("No se pudo procesar el archivo. Revisa que la ruta sea correcta.")

print("="*60)

## Aplicación 2: Compresor de Código Fuente (.py)

Una aplicación interesante y distinta es la compresión de **código fuente**. Aunque el código también es texto, su distribución de caracteres es muy diferente a la del lenguaje natural.

En el código fuente (por ejemplo, de Python) son muy frecuentes caracteres como los **espacios de indentación, paréntesis `()`, puntos `.`, comas `,` y saltos de línea**. En cambio, la variedad de letras del alfabeto puede ser menor o tener una distribución diferente.

Esta estructura particular hace que el código fuente sea un buen candidato para la compresión con Huffman. A continuación, comprimiremos el propio motor de compresión de este proyecto para evaluar el rendimiento.

In [ ]:
# --- Aplicación 2: Compresión de un archivo de código fuente ---

# Definimos la ruta del archivo de código.
ruta_codigo_fuente = '../data/config.py'
# El archivo .py que se utilizó fue sacado de el repositorio oficial de Django

print("="*60)
print("APLICACIÓN 2: ANÁLISIS DE COMPRESIÓN DE CÓDIGO FUENTE")
print("="*60)

# Analizamos la compresión del archivo de código.
resultado_codigo = analizar_compresion(ruta_codigo_fuente)

if resultado_codigo:
    # Imprimimos los resultados.
    print(f"Archivo Analizado:   {resultado_codigo['archivo']}")
    print(f"Tamaño Original:      {resultado_codigo['original_b']} bytes")
    print(f"Tamaño Comprimido:    {resultado_codigo['total_comprimido_b']} bytes")
    print(f"Ratio de Compresión:  {resultado_codigo['ratio_%']:.2f}%")
else:
    print("No se pudo procesar el archivo. Revisa que la ruta sea correcta.")

print("="*60)

## Aplicación 3: Compresión de Secuencias de ADN (.fasta, .txt)

Una de las aplicaciones más impactantes de la compresión de datos se encuentra en la **bioinformática**, específicamente en el almacenamiento de secuencias genómicas. El ADN se representa con un alfabeto increíblemente pequeño, compuesto casi en su totalidad por cuatro bases nitrogenadas: **Adenina (A), Citosina (C), Guanina (G) y Timina (T)**.

Este escenario es ideal para la codificación de Huffman. Al tener un conjunto tan limitado y repetitivo de caracteres, el algoritmo puede asignar códigos binarios extremadamente cortos a estas cuatro letras, logrando **ratios de compresión muy elevados**.

En un campo donde se generan terabytes de datos genómicos, la capacidad de comprimir eficientemente esta información es fundamental para el almacenamiento y la transmisión de datos en bases de datos a nivel mundial. A continuación, se demuestra esta capacidad comprimiendo un archivo que simula una secuencia de ADN.

In [ ]:

# Este archivo contiene una secuencia larga de los caracteres A, C, G, T.
ruta_adn = '../data/adn.txt'

print("="*60)
print("APLICACIÓN 3: ANÁLISIS DE COMPRESIÓN DE SECUENCIA DE ADN")
print("="*60)

# Utilizamos la misma función de análisis de siempre.
resultado_adn = analizar_compresion(ruta_adn)

if resultado_adn:
    # Imprimimos los resultados.
    print(f"Archivo Analizado:   {resultado_adn['archivo']}")
    print(f"Tamaño Original:      {resultado_adn['original_b']} bytes")
    print(f"Tamaño Comprimido:    {resultado_adn['total_comprimido_b']} bytes")
    print(f"Ratio de Compresión:  {resultado_adn['ratio_%']:.2f}%")
    print("\nConclusión: Como se esperaba, el alfabeto reducido del ADN permite")
else:
    print("No se pudo procesar el archivo. Asegúrate de que el archivo 'adn.txt' exista en la carpeta /data.")

print("="*60)

APLICACIÓN 3: ANÁLISIS DE COMPRESIÓN DE SECUENCIA DE ADN
Archivo Analizado:   adn.txt
Tamaño Original:      1000000 bytes
Tamaño Comprimido:    250265 bytes
Ratio de Compresión:  74.97%

Conclusión: Como se esperaba, el alfabeto reducido del ADN permite
un ratio de compresión excepcionalmente alto, demostrando la gran
utilidad de Huffman en el campo de la bioinformática.


# Experimentos
## Experimento 1: Pruebas en 3 escenarios
Se pone a prueba el algoritmo de codificación de mensaje en tres casos: un texto promedio (el inicio del libro "Don Quijote de la mancha"), un texto con los mismos 10 caracteres repetidos y por último un texto con caracteres aleatorios.
En los tres casos se busco igualar la cantidad de caracteres a aproximadamente 10351, que es una cifra dispuesta de manera arbitraria. De manera que todos tienen aproximadamente la misma cantidad de caracteres y en lo que difieren es en la frecuencia y el rango o variedad de estos. 

In [ ]:
archivos_a_probar = [
        '../data/quijote.txt',
        '../data/texto_repetitivo.txt',
        '../data/texto_aleatorio.txt'
    ]

if __name__ == "__main__":
    results_exp1 = obtener_resultados(archivos_a_probar)
    if results_exp1:
        crear_grafico_comparativo(results_exp1)
        print("Gráfico guardado en la carpeta /results/")
    else:
        print("No hay resultados que mostrar")


## Experimento 2: Escalabilidad del Algoritmo

En este segundo experimento, el objetivo es analizar la **escalabilidad y la eficiencia** del algoritmo de Huffman a medida que aumenta el tamaño de los datos de entrada. Se busca observar cómo se comporta el ratio de compresión cuando los archivos a procesar son progresivamente más grandes.

Para ello, se generan archivos de prueba de distintos tamaños (desde 1 KB hasta 100 KB) para cada uno de los tres tipos de texto: normal (basado en el Quijote), altamente repetitivo y aleatorio. Posteriormente, se grafica el tamaño original contra el tamaño comprimido para cada categoría. Esto nos permite visualizar de manera clara cómo escala la compresión y si la naturaleza de los datos influye en su rendimiento a mayor escala.

In [ ]:
def visualizar_escalabilidad():
    tipos_de_texto = ['normal', 'repetitivo', 'aleatorio']
    tamaños_kb = [1, 5, 10, 25, 50, 100] # Tamaños en Kilobytes
    

    for tipo in tipos_de_texto:
        preparar_datos_escalabilidad(tipo, tamaños_kb)


    # Lista para guardar todos los resultados
    resultados_totales = []
    
    for tipo in tipos_de_texto:
        for kb in tamaños_kb:
            ruta = f'../data/temp/{tipo}_{kb}kb.txt'
            resultado = analizar_compresion(ruta)
            if resultado:
                resultado['Tipo'] = tipo # Añadir el tipo para agrupar después
                resultados_totales.append(resultado)
    
    if not resultados_totales:
        print("No se generaron resultados. Verifica las rutas de los archivos.")
        return

    # Convertir a df 
    df_final = pd.DataFrame(resultados_totales)
    graficar_escalabilidad(df_final)



# Ejecutar el nuevo experimento
visualizar_escalabilidad()